In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 라이브러리 임포트
import os
import time
import json
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
#from fbprophet import Prophet
import pickle
import copy
from tqdm.auto import tqdm

In [3]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: user 765 ms, sys: 130 ms, total: 895 ms
Wall time: 896 ms


In [4]:
''' fill na '''
for plc in plc_lst:
    df = df_dic[plc]
    df_dic[plc] = df_dic[plc].fillna(method='ffill')
    df_dic[plc] = df_dic[plc].fillna(method='bfill')
    df_dic[plc] = df_dic[plc].dropna()

In [6]:
%%time
''' create datetime '''
for plc in plc_lst:
    df_dic[plc]['date'] = df_dic[plc]['ds'].apply(lambda x : datetime.strptime(x , '%Y%m%d'))
    df_dic[plc]['year'] = pd.DatetimeIndex(df_dic[plc]['date']).year - 2010
    df_dic[plc]['month'] = pd.DatetimeIndex(df_dic[plc]['date']).month / 12
    df_dic[plc]['day'] = pd.DatetimeIndex(df_dic[plc]['date']).day / 31

CPU times: user 6.81 s, sys: 3.55 ms, total: 6.82 s
Wall time: 6.82 s


---
# 데이터 만들기

In [7]:
def create_lag_feature(df_trg , lag_count , interval):
    df_n = df_trg.copy()
    for n in range(1, lag_count + 1):
        for col in fct_lst:
            df_n[f"{col}_lag{n*interval}"] = df_n[col].shift(n*interval)
    dftemp = df_n.iloc[n_lags*interval:].dropna()
    #dftemp = df_n.iloc[n_lags*interval:-1]
    y = dftemp[fct_lst].shift(-1).dropna()
    x = dftemp.iloc[:-1].drop(['ds','date'],axis = 1)
    return x, y

In [8]:
%%time
lag_count = 120
interval = 5

xs = dict()
ys = dict()
for plc in plc_lst:
    x ,y = create_lag_feature(df_dic[plc] , lag_count , interval)
    xs[plc] = x
    ys[plc] = y

/tmp/ipykernel_5399/3529126758.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_n[f"{col}_lag{n*interval}"] = df_n[col].shift(n*interval)
/tmp/ipykernel_5399/3529126758.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_n[f"{col}_lag{n*interval}"] = df_n[col].shift(n*interval)
/tmp/ipykernel_5399/3529126758.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

NameError: name 'n_lags' is not defined

In [77]:
xs[plc].columns

Index(['pH', 'COD', 'SS', 'N', 'P', 'T', 'year', 'month', 'day', 'pH_lag5',
       ...
       'SS_lag595', 'N_lag595', 'P_lag595', 'T_lag595', 'pH_lag600',
       'COD_lag600', 'SS_lag600', 'N_lag600', 'P_lag600', 'T_lag600'],
      dtype='object', length=729)